In [3]:
import numpy as np
import time 
import causaltensor as ct
from causaltensor import low_rank_M0_Gamma
from causaltensor.matlib import generate_Z
import causaltensor.cauest.DebiasConvex as DC
import pandas as pd
import pickle

## Semi-synthetic experiments on Sales data

In [67]:
M0 = pickle.load(open('sales.p', 'rb'))
s = np.linalg.svd(M0, full_matrices=False, compute_uv=False)

from importlib import reload
reload(DC)

def experiments_run(M0, num_experiment=1, sigma = 0.1):
    results = 0
    for T in range(num_experiment):
        while True:
            a = np.random.randint(20)+5
            b = np.random.randint(20)+5
            Z, info = generate_Z(pattern_tuple = ['adaptive', (a, b)], M0=M0)
            if (info == 'fail'):
                continue
            break
        tau_star = np.mean(M0)/5
        E = np.random.normal(loc=0, scale=sigma, size=M0.shape)
        O = M0 + tau_star * Z + E   
        M, tau, std = DC.DC_PR_auto_rank(O, Z)
        results += np.linalg.norm(tau-tau_star) / np.linalg.norm(tau_star)
    return results / num_experiment    

np.random.seed(0)
error = experiments_run(M0, num_experiment=10, sigma = 0)

In [68]:
assert error < 0.1

In [69]:
error

0.024746699133216143